<a href="https://colab.research.google.com/github/aayushkubb/nlp/blob/main/Classification_using_Transfomers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/Quora/QuoratrainSet.csv')
test=pd.read_csv('/content/drive/MyDrive/Quora/Quoratestdata.csv')

# Zero Shot Models

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
text = "The company's quarterly earnings increased by 20%, exceeding market expectations."
candidate_labels = ["finance", "sports", "politics", "technology"]

result = pipe(text, candidate_labels)

print(result)


{'sequence': "The company's quarterly earnings increased by 20%, exceeding market expectations.", 'labels': ['finance', 'technology', 'sports', 'politics'], 'scores': [0.6282327175140381, 0.2245805412530899, 0.08779522031545639, 0.05939146503806114]}


In [6]:
train['question_text'][10]

'What can you say about feminism?'

In [8]:
text = train['question_text'][10]
candidate_labels = ["Sincere Question", "InSincere Question"]

result = pipe(text, candidate_labels)

print(result)


{'sequence': 'What can you say about feminism?', 'labels': ['Sincere Question', 'InSincere Question'], 'scores': [0.6316617727279663, 0.3683382272720337]}


In [17]:
text=train[train['target']==1].sample(1)['question_text'].tolist()[0]
print(text)

candidate_labels = ["Sincere Question", "InSincere Question"]

result = pipe(text, candidate_labels)

print(result)


What is the difference between a computer with AI superintelligence but no emotion, and a psychopath?
{'sequence': 'What is the difference between a computer with AI superintelligence but no emotion, and a psychopath?', 'labels': ['Sincere Question', 'InSincere Question'], 'scores': [0.5600026845932007, 0.43999725580215454]}


# Few shot Learning -1

In [24]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cpu


In [20]:
few_shot_examples = []
for label in [0, 1]:
    examples = train[train['target'] == label].sample(10)['question_text'].tolist()
    for example in examples:
        few_shot_examples.append({'text': example, 'label': label})

In [21]:
few_shot_examples

[{'text': 'How do I control my Quora addiction?', 'label': 0},
 {'text': "Why does barium sulphate doesn't soluble in acids?", 'label': 0},
 {'text': 'What is the full form of DOD?', 'label': 0},
 {'text': 'What is the derivative of √2x^3 +4?', 'label': 0},
 {'text': 'Which class are you going to main at the start of World of Warcraft: Battle for Azeroth?',
  'label': 0},
 {'text': 'If you were a character of a comedy series made on your life, what does the trailer say? (describe in your words)',
  'label': 0},
 {'text': 'Then why India today is poor?', 'label': 0},
 {'text': 'What is the best book for WB psc ae civil?', 'label': 0},
 {'text': 'What is different between software and hardware?', 'label': 0},
 {'text': 'How is the PCM faculty of CMS Gomti Nagar?', 'label': 0},
 {'text': 'Why are all of the grocery stores in Belize owned by Chinese? Is it a policy towards taking over the country?',
  'label': 1},
 {'text': 'If we moved all Caucasians to Asia and all Asians to Western Euro

In [28]:
text=train[train['target']==1].sample(1)['question_text'].tolist()[0]
candidate_labels = ["Sincere Question", "Insincere Question"]

result = classifier(text, candidate_labels, hypothesis_template="This example is a {}.")

print(result)

{'sequence': 'How do I fuck some bitches?', 'labels': ['Sincere Question', 'Insincere Question'], 'scores': [0.8312954902648926, 0.16870443522930145]}


# Few Shot Learning -2

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from datasets import Dataset

train_df=pd.read_csv('/content/drive/MyDrive/Quora/QuoratrainSet.csv')
test_df=pd.read_csv('/content/drive/MyDrive/Quora/Quoratestdata.csv')

In [4]:
model_name = "bert-base-uncased"  # Or another suitable model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2 labels: sincere/insincere

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
few_shot_examples = []
for label in [0, 1]:
    examples = train_df[train_df['target'] == label].sample(10)['question_text'].tolist()
    for example in examples:
        few_shot_examples.append({'text': example, 'label': label})

NameError: name 'train_df' is not defined

In [6]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_examples = [preprocess_function({'text': example['text']}) for example in few_shot_examples]

# Add labels to tokenized examples
for i, example in enumerate(tokenized_examples):
    example['labels'] = few_shot_examples[i]['label']

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,  # Adjust as needed
    num_train_epochs=3,  # Adjust as needed
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_examples,  # Use tokenized few-shot examples
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aayushkubba-727 (aayushkubba-727-openai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss


TrainOutput(global_step=9, training_loss=0.6133447223239474, metrics={'train_runtime': 174.3933, 'train_samples_per_second': 0.344, 'train_steps_per_second': 0.052, 'total_flos': 15786663321600.0, 'train_loss': 0.6133447223239474, 'epoch': 3.0})

In [22]:
new_question = train_df[train_df['target']==0].sample(1)['question_text'].tolist()[0]
print(new_question)
inputs = tokenizer(new_question, return_tensors="pt")

# Move inputs to the same device as the model
device = next(model.parameters()).device  # Get model's device
inputs = inputs.to(device)  # Move inputs to the device

outputs = model(**inputs)

predicted_class = outputs.logits.argmax().item()  # 0 for sincere, 1 for insincere

print(f"Predicted class: {predicted_class}")

Which is the best Samsung Printer support Service in the US?
Predicted class: 0


# Using LLMs

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [2]:
from transformers import pipeline

In [ ]:
generator = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def classify_question(your_question):
    prompt = f"""
    Classify the following question as either "Sincere Question" or "Insincere Question".

    An insincere question is defined as a question intended to make a statement rather than look for helpful answers. Some characteristics that can signify that a question is insincere:

    -Has a non-neutral tone
    -Has an exaggerated tone to underscore a point about a group of people
    -Is rhetorical and meant to imply a statement about a group of people
    -Is disparaging or inflammatory
    -Suggests a discriminatory idea against a protected class of people, or seeks confirmation of a stereotype
    -Makes disparaging attacks/insults against a specific person or group of people
    -Based on an outlandish premise about a group of people
    -Disparages against a characteristic that is not fixable and not measurable
    -Isn't grounded in reality
    -Based on false information, or contains absurd assumptions
    -Uses sexual content (incest, bestiality, pedophilia) for shock value, and not to seek genuine answers

    ## Examples:

    **Sincere Question:** What are the best resources for learning Python?
    **Insincere Question:** Why are all politicians so incompetent?
    **Sincere Question:** How do I bake a chocolate cake?
    **Insincere Question:** Are all liberals snowflakes?

    ## Question to Classify:

    {your_question}
    """

    result = generator(prompt, max_new_tokens=10)
    prediction = result[0]['generated_text'].strip()
    return prediction

In [8]:
classify_question("What are the best resources for learning Python?")

NameError: name 'generator' is not defined

In [ ]:
classify_question("Why are all politicians so incompetent?")